In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import string
from random import randint
from time import sleep
import random
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Heiti TC']
from PIL import ImageTk as itk
import tkinter as tk

In [21]:
def get_df(char):
    URL = "http://www.taiwantestcentral.com/wordlist/WordListByName.aspx?MainCategoryID=25&Letter={}".format(char)
    
    HEADERS = ({'User-Agent':
                'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 \
                (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',\
                'Accept-Language': 'en-US, en;q=0.5'})
    
    webpage = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")

    table = soup.find('table', {'class': 'WordList fg_highlight'})
    ###
    columns = [th.text.replace('\n', '') for th in table.find('tr').find_all('th')]

    trs = table.find_all('tr')[1:]
    rows = list()
    for tr in trs:
        rows.append([td.text.replace('\n', '').replace('\xa0', '') for td in tr.find_all('td')])
    ###
    df = pd.DataFrame(data=rows, columns=columns)
    df = df.drop('考題', axis=1)
    df = df.rename(columns={'級別': 'classification', '字詞': 'vocab', '中文釋義':'chinese'})
    return df

In [ ]:
df = get_df('A')
for i in list(string.ascii_uppercase)[1:]:
    sleep(randint(1,10))
    df = df.append(get_df(i), ignore_index=True)

df = df.drop('', axis=1)
df['chinese'] = df['chinese'].str.split('; ')
df = df.explode('chinese')
df[['pos', 'def']] = df['chinese'].str.split('] ', 1, expand=True)
df['pos'] = df['pos'].str.replace('[', '', regex=True)
df.to_csv('hs_voc.csv', encoding='utf-8-sig', columns=['classification', 'vocab', 'pos', 'def'])

In [4]:
df = pd.read_csv('hs_voc.csv', index_col=0)
df['classification'] = df['classification'].replace(['附\u3000錄'],'附錄')
item_dict = {k: [g["def"].tolist(), list(set(g["classification"].tolist()))] for k,g in df.groupby("vocab")}

In [5]:
def change_img():
    canvas.delete("all")
    canvas.img = itk.PhotoImage(file= 'new_output.png')
    canvas.create_image(30, 10, anchor=tk.NW, image=canvas.img)

def button_event():
    voc_list = random.choices(list(item_dict.keys()), k=10)
    class_list = [item_dict[key][1][0] for key in voc_list]
    def_list = [item_dict[key][0][0] for key in voc_list]
    df = pd.DataFrame({'voc': voc_list, 'class':class_list, 'def':def_list })
    ax = plt.axes(frame_on=False)# 不要額外框線
    ax.xaxis.set_visible(False)  # 隱藏X軸刻度線
    ax.yaxis.set_visible(False)  # 隱藏Y軸刻度線
    ax.axis('tight')
    ax.axis('off')
    tmp = pd.plotting.table(ax, df, loc='center')
    tmp.auto_set_font_size(False)
    tmp.set_fontsize(10)
    plt.savefig('new_output.png')
    root.after(5000, change_img)

In [6]:
root = tk.Tk()
root.title('my window')
root.geometry('500x350')

mybutton = tk.Button(root, text='Send', command=button_event)
mybutton.pack()

photo = itk.PhotoImage(file= 'tmp.png')
canvas = tk.Canvas(root, width=700, height=700)
canvas.imageList = []
canvas.pack()
canvas.create_image(80, 0, anchor=tk.NW, image=photo)
canvas.imageList.append(photo)

root.mainloop()